In [ ]:
!pip install spacy nltk pandas tqdm
# try to download a small multilingual model if available (no-op if missing)
!python -m spacy download xx_ent_wiki_sm || true
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
try:
    nlp = spacy.load("xx_ent_wiki_sm")
except Exception:
    nlp = spacy.blank("xx")
import inflect

In [ ]:
# Keep inflect for API compatibility (Telugu doesn't use English articles)
p = inflect.engine()

In [ ]:
COMPLEX_PATTERNS = {
  "premises": {
    "all": [
      {"tpl": "అన్ని {a_pl} లు {b_pl}.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ప్రతి {a_sg} ఒక {b_sg}.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "అన్ని {a_pl} లు ఎటువంటి తప్పు లేకుండా {b_pl} గా ఉన్నాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} సాధారణంగా {b_pl} గా వర్గీకరించబడతాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లను {b_pl} గా పరిగణిస్తారు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ప్రతి ఒక్క {a_sg} ఖచ్చితంగా {b_sg} వర్గానికి చెందుతుంది.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "మినహాయింపు లేకుండా, {a_pl} అన్నీ {b_pl} లు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} యొక్క మొత్తం సమూహం {b_pl} కిందకు వస్తుంది.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ఏదైనా {a_sg} తీసుకున్నా, అది తప్పనిసరిగా {b_sg} అవుతుంది.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "{a_pl} అనేవి ఎల్లప్పుడూ {b_pl} గానే గుర్తించబడతాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} అన్నీ {b_pl} లో అంతర్భాగమే.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "లోకంలో ఉన్న అన్ని {a_pl} లు {b_pl} వర్గానికి చెందినవే.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ఒకవేళ అది {a_sg} అయితే, అది ఖచ్చితంగా {b_sg}.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "{a_pl} అన్నవి ముమ్మాటికీ {b_pl} లే.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_sg} ప్రతిదీ {b_sg} గానే పరిగణించబడుతుంది.", "a_type": "singular", "b_type": "singular"}
    ],
    "some": [
      {"tpl": "కొన్ని {a_pl} {b_pl}.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో కొన్ని {b_pl} ఉన్నాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ఒక భాగం {a_pl} {b_pl}.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొంతమంది {a_pl} {b_pl} గానే ఉంటారు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కనీసం కొన్ని {a_pl} నిజంగా {b_pl}.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} సమూహంలో కొన్ని {b_pl} లు కనపడతాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "అరుదుగానైనా కొన్ని {a_pl} {b_pl} గా ఉంటాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "చాలా వరకు కాకపోయినా, కొన్ని {a_pl} {b_pl} లే.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో కొన్ని మాత్రమే {b_pl} కిందకు వస్తాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొన్ని సందర్భాల్లో {a_pl} {b_pl} గా మారుతాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "తక్కువ సంఖ్యలోనైనా {a_pl} {b_pl} గా గుర్తించబడ్డాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "నిర్దిష్టంగా చెప్పాలంటే కొన్ని {a_pl} {b_pl} వర్గానికి చెందుతాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొన్ని {a_pl} లు {b_pl} కి అన్వయిస్తాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "సమయం ప్రకారం కొన్ని {a_pl} {b_pl} గా ఉండవచ్చు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొద్దిపాటి {a_pl} {b_pl} కి నిదర్శనం.", "a_type": "plural", "b_type": "plural"}
    ],
    "no": [
      {"tpl": "ఏ {a_pl} కూడా {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "అందులో ఏ {a_pl} అయినా {b_pl} ఉండదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} మరియు {b_pl} కు పరిధి లేదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "వాస్తవానికి, ఎటువంటి {a_pl} {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} పూర్తిగా {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "అసలు ఏ {a_sg} కూడా {b_sg} అయ్యే అవకాశం లేదు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "ఖచ్చితంగా చెప్పాలంటే {a_pl} ఏవీ {b_pl} కావు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} వర్గానికి మరియు {b_pl} వర్గానికి ఎటువంటి సంబంధం లేదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ఒకవేళ అది {a_sg} అయితే, అది {b_sg} ఎప్పటికీ కాదు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "{a_pl} అన్నీ {b_pl} కి వ్యతిరేకం.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_sg} ఎన్నడూ {b_sg} కాలేదు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "ఏ పరిస్థితుల్లోనూ {a_pl} {b_pl} కావు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో ఒక్కటి కూడా {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_sg} మరియు {b_sg} మధ్య అసలు పొంతన లేదు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "మొత్తంగా చూస్తే ఏ {a_pl} కూడా {b_pl} జాబితాలో లేవు.", "a_type": "plural", "b_type": "plural"}
    ],
    "some_not": [
      {"tpl": "కొన్ని {a_pl} {b_pl} కావు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో కొన్ని భాగాలు {b_pl} కాదేమో.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "భాగం-wise, కొన్ని {a_pl} {b_pl} కాకపోవచ్చు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొన్ని {a_pl} ఖచ్చితంగా {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "అందరూ {a_pl} {b_pl} కాదు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ప్రతి ఒక్క {a_sg} {b_sg} కానవసరం లేదు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "కొన్ని {a_pl} లు {b_pl} జాబితా నుండి మినహాయించబడ్డాయి.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో చాలా వరకు {b_pl} అయినా, కొన్ని కావు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "అన్ని {a_pl} లూ {b_pl} కాకపోవడం గమనార్హం.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "నిజానికి కొన్ని {a_pl} {b_pl} వర్గానికి వ్యతిరేకం.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "{a_pl} లో కనీసం ఒకటి {b_pl} కాదని తెలుస్తోంది.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొన్ని {a_pl} లు {b_pl} లక్షణాలను కలిగి లేవు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "ఖచ్చితంగా కొన్ని {a_sg} లు {b_sg} కావు.", "a_type": "singular", "b_type": "singular"},
      {"tpl": "{a_pl} అన్నీ {b_pl} అని అనుకోలేము, కొన్ని కావు.", "a_type": "plural", "b_type": "plural"},
      {"tpl": "కొంతమేరకు {a_pl} {b_pl} కాదని స్పష్టమవుతోంది.", "a_type": "plural", "b_type": "plural"}
    ]
  },
  "conclusions": [
    "కాబట్టి, {conclusion}.",
    "ఆ కారణంగా, {conclusion}.",
    "అందువల్ల, {conclusion}.",
    "ఈ సందర్భంలో, {conclusion}.",
    "దీనిని బట్టి మనం చెప్పవచ్చు: {conclusion}.",
    "తుది ఫలితంగా, {conclusion}.",
    "నిర్ణయాత్మకంగా చూస్తే, {conclusion}.",
    "మేము దీని ద్వారా గ్రహించవచ్చు: {conclusion}.",
    "మొత్తం మీద, {conclusion}.",
    "దీని సారాంశం ఏమిటంటే, {conclusion}."
  ]
}

In [ ]:
COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
  "premises": {
    "all": [
      {"tpl": "అన్ని {d_pl} వారిలోని {a_pl} కూడా {b_pl}.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ప్రతి {d_sg} అది {a_sg} అయితే అది {b_sg} కూడా.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "ఏ {d_sg} అయినా అది {a_sg} మరియు కాదు {b_sg} అనే పరిస్థితి లేదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "ఎవరైతే {d_sg} మరియు {a_sg} అవుతారో వారు తప్పకుండా {b_sg}.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl}లోని అన్ని {a_pl}లు {b_pl}లో పడతాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఒకవేళ ఏదయినా {d_sg} {a_sg} గా ఉంటే, అది ఖచ్చితంగా {b_sg} వర్గానికి చెందుతుంది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "మినహాయింపు లేకుండా, {a_pl} మరియు {d_pl} రెండూ అయిన ప్రతిదీ {b_pl} అవుతుంది.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{a_pl} గా గుర్తించబడిన ప్రతి {d_sg} ని {b_sg} గానే భావించాలి.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} సమూహంలో ఏవేవైతే {a_pl} అవుతాయో, అవి ముమ్మాటికీ {b_pl} లే.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఏ {d_sg} అయితే {a_sg} కిందకు వస్తుందో, అది ఎల్లప్పుడూ {b_sg} గానే ఉంటుంది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} మరియు {a_pl} మధ్య సామాన్యంగా ఉన్నవన్నీ {b_pl} లో అంతర్భాగమే.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఏదైనా వస్తువు {d_sg} మరియు {a_sg} అయితే, అది తప్పక {b_sg} అవుతుంది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} జాబితాలోని ప్రతి {a_pl} ని {b_pl} గానే పరిగణనలోకి తీసుకోవాలి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{a_pl} అనే గుణం ఉన్న ప్రతి {d_sg} కి {b_sg} అనే గుణం కూడా ఉంటుంది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "సాధారణంగా {d_pl} లో ఉండి {a_pl} గా పిలువబడేవన్నీ {b_pl} కు నిదర్శనాలు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"}
    ],
    "some": [
      {"tpl": "కనీసం ఒక {d_sg} ఉంది, అది {a_sg} మరియు {b_sg}.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "కొన్ని {d_pl}‌లు {a_pl} మరియు {b_pl} గా ఉంటాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "కొన్ని సందర్భాల్లో, {d_sg} {a_sg} కూడా {b_sg}.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "భాగంగా ఉన్న {d_pl}లో కొన్ని {a_pl} మరియు {b_pl} గణించబడతాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "చాలా కొద్దిగా {d_sg}‌లు {a_sg} మరియు {b_sg}గా ఉంటాయి.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} సమూహంలో కొన్ని {a_pl} లు కూడా {b_pl} గానే గుర్తించబడ్డాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "నిజానికి కొన్ని {d_sg} లు {a_sg} మరియు {b_sg} లక్షణాలను కలిగి ఉన్నాయి.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "అరుదైన సందర్భాలలో {d_pl} లో ఉండి {a_pl} మరియు {b_pl} రెండూ అయినవి ఉంటాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{d_pl} జాబితాలో కొన్ని {a_pl} లు మరియు {b_pl} లు ఒకేసారి కనపడతాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఒక నిర్దిష్ట {d_sg} ని తీసుకుంటే, అది {a_sg} మరియు {b_sg} కూడా కావచ్చు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} లో ఉన్న కొద్దిపాటి {a_pl} లు {b_pl} వర్గానికి చెందుతున్నాయి.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "కనీసం కొన్ని సార్లు {d_sg} అనేది {a_sg} గా ఉంటూనే {b_sg} గా కూడా ఉంటుంది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{a_pl} మరియు {b_pl} రెండూ అయిన {d_pl} లో కొన్ని ఉన్నాయని స్పష్టమవుతోంది.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "మనం గమనిస్తే, కొన్ని {d_sg} లు ఏకకాలంలో {a_sg} మరియు {b_sg} గా ఉన్నాయి.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} వర్గంలో కొన్ని {a_pl} ని, {b_pl} ని కలిగి ఉన్నట్లు కనుగొన్నాము.", "a_type": "plural", "b_type": "plural", "d_type": "plural"}
    ],
    "no": [
      {"tpl": "{d_pl}లో ఎవరూ {a_pl} మరియు {b_pl} కారు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఏ {d_sg} అయినా అది {a_sg} మరియు {b_sg} కాదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl}లో ఎప్పుడూ {a_pl} మరియు {b_pl} ఒకే వారి వద్ద ఉండరు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "యాదృచ్ఛికంగా, {d_sg} అయినా అది {a_sg} అయితే అది {b_sg} కాదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "ప్రత్యేకంగా, ఎలాంటి {d_sg} {a_sg} అయినా అది {b_sg} కాదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} లో {a_pl} మరియు {b_pl} ల మధ్య ఎటువంటి పొంతన లేదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "అసలు ఏ {d_sg} కూడా ఏకకాలంలో {a_sg} మరియు {b_sg} అయ్యే అవకాశం లేదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} వర్గంలో ఉన్నవన్నీ చూస్తే, ఒక్కటి కూడా {a_pl} మరియు {b_pl} రెండూ కాదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఒకవేళ అది {d_sg} మరియు {a_sg} అయితే, అది ఎప్పటికీ {b_sg} కాబోదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} జాబితాలో {a_pl} లు ఏవీ కూడా {b_pl} కిందకు రావు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{d_sg} అనేది ఒకవేళ {a_sg} అయితే, అది {b_sg} అవ్వడం అసాధ్యం.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "ఖచ్చితంగా చెప్పాలంటే, {d_pl} లో ఉండి {a_pl} మరియు {b_pl} రెండూ అయినవి ఏవీ లేవు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{d_sg} కి {a_sg} అనే గుణం ఉంటే, దానికి {b_sg} అనే గుణం ఉండదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} లోని ఏ ఒక్కటి కూడా {a_pl} మరియు {b_pl} లక్షణాలను కలిసి కలిగి లేదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{a_pl} మరియు {b_pl} ల మధ్య {d_pl} కు ఎటువంటి సంబంధం లేదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"}
    ],
    "some_not": [
      {"tpl": "అన్నీ {d_sg}లలో {a_sg} అయినా అవి {b_sg} కావు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "ఎక్కడో ఒక {d_sg} ఉంది, అది {a_sg} కానీ కాదు {b_sg}.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "కొన్ని {d_pl}గా ఉన్నాయి, అవి {a_pl} కానీ {b_pl} కాదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "కొన్ని {d_sg}లు {a_sg} గానే ఉంటాయి కానీ {b_sg} అవ్వవచ్చు కాదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "భాగంగా ఉన్న కొన్ని {d_pl}లు {a_pl} కానీ కాదు {b_pl}.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "{d_pl} లోని అన్ని {a_pl} లు {b_pl} కావు అని స్పష్టమవుతోంది.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "కనీసం ఒక {d_sg} ఉంది, అది {a_sg} అయినప్పటికీ {b_sg} కాదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "కొన్ని {d_pl} లు {a_pl} లక్షణాలను కలిగి ఉన్నా, {b_pl} వర్గానికి చెందవు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "నిజానికి ప్రతి {d_sg} {a_sg} అయినప్పటికీ, అన్నింటికీ {b_sg} అనే గుణం ఉండదు.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} లో ఉండి {a_pl} గా పిలవబడే వాటిలో కొన్ని {b_pl} జాబితాలో లేవు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "కొంతమంది {d_pl} {a_pl} గా గుర్తింపబడ్డా, వారు {b_pl} కాదు.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "ఒక నిర్దిష్ట {d_sg} {a_sg} అని తెలిసినా, అది {b_sg} కాదని తేలింది.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{d_pl} సమూహంలో కొన్ని {a_pl} లు {b_pl} కాకపోవడం గమనార్హం.", "a_type": "plural", "b_type": "plural", "d_type": "plural"},
      {"tpl": "తక్కువ సంఖ్యలోనైనా కొన్ని {d_sg} లు {a_sg} గా ఉంటూ {b_sg} కావని గుర్తించబడ్డాయి.", "a_type": "singular", "b_type": "singular", "d_type": "singular"},
      {"tpl": "{a_pl} అయిన ప్రతి {d_sg} కి {b_sg} అనే గుణం ఉంటుందని మనం అనుకోలేము.", "a_type": "singular", "b_type": "singular", "d_type": "singular"}
    ]
  }
}

In [ ]:
def get_singular_and_plural(term):
    term = term.strip()
    # Telugu templates use same token for compatibility
    return term, term

In [ ]:
def get_article(term):
    # Telugu doesn't use 'a'/'an'; return term unchanged
    return term.strip()

In [ ]:
def get_basic_syllogism_parts(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    return sentences

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    match = re.match(r"All (.*) that are (.*) are (.*).", sentence)
    if match: return "all", match.group(1), match.group(2), match.group(3)
    match = re.match(r"Some (.*) that are (.*) are not (.*).", sentence)
    if match: return "some_not", match.group(1), match.group(2), match.group(3)
    match = re.match(r"Some (.*) that are (.*) are (.*).", sentence)
    if match: return "some", match.group(1), match.group(2), match.group(3)
    match = re.match(r"No (.*) that are (.*) are (.*).", sentence)
    if match: return "no", match.group(1), match.group(2), match.group(3)
    match = re.match(r"All (.*) are (.*).", sentence)
    if match: return "all", None, match.group(1), match.group(2)
    match = re.match(r"Some (.*) are not (.*).", sentence)
    if match: return "some_not", None, match.group(1), match.group(2)
    match = re.match(r"Some (.*) are (.*).", sentence)
    if match: return "some", None, match.group(1), match.group(2)
    match = re.match(r"No (.*) are (.*).", sentence)
    if match: return "no", None, match.group(1), match.group(2)
    raise ValueError(f"Could not parse basic syllogism type from sentence: {sentence}")

In [ ]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f"Syllogism must contain at least one premise and one conclusion: {syllogism}")
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        try:
            p_complex = p_complex[0].upper() + p_complex[1:]
        except Exception:
            pass
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1]
    try:
        c_complex = c_complex[0].upper() + c_complex[1:]
    except Exception:
        pass
    final_complex_syllogism = " ".join(complex_premises) + " " + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f\
            continue
    return complex_dataset

In [ ]:
# Example Data for Testing (input stays symbolic/simple English form)
sample_data = [
  {"id": "tel-1", "syllogism": "All animals that are non-aquatic are fish. Some mammals are non-aquatic. Some mammals are not fish.", "validity": True, "plausibility": True},
  {"id": "tel-2", "syllogism": "All υ are Π. All Π are Ε. Some Ε are υ.", "validity": True, "plausibility": True},
  {"id": "tel-3", "syllogism": "No fish are mammals. Some mammals are whales. Some whales are not fish.", "validity": True, "plausibility": True},
]
complex_syllogism_data = process_dataset(sample_data, COMPLEX_PATTERNS, COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print("
--- మార్పు చేసిన క్లిష్టమైన శిలాజిజమ్స్ (Telugu) ---")
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))